# Project Report (in progress)
##### Michael Chu, Junyu Wang, Yinghao Li

## Intro

A lot of portfolio optimization algorithms run into practical difficulties like numerical instability and sensitivity to perturbations/noisy data when the list of assets is too large. For example, Markowitz relies on an estimation of asset returns (very noisy and hard to estimate) and its covariance. While covariance estimate is relatively more robust, Markowitz computation requires an inversion of the said covariance matrix, which will almsot certainly run into numerical issues, such as violation of P.S.D. or other necessary properties of valid correlations. 

We attempt to solve this problem by curating and filtering through a superset of assets using graph sampling methods.

A paper from Pozzi et al. provides an interesting Graph-theoretic interpretation of asset return correlations. They also boast some good results from backtesting their theory . We thought* the paper sounded promising, so we take inspiration from their methods and try it out ourselves.

## Correlation Matrix as Network

Suppose we have a set of assets that we want to consider. Naturally we can compute its daily returns (log or arithmetic) and its covariance matrix and correlation matrix.etc. We consider the correlation matrix (minus the diagonal elements) of returns as the adjacency matrix of a graph/network. We apply some graph filtering methods to cut down on the number of edges/connections within the network, while stlil capturing the underlying structure of such network. 

Examples of Filtering methods:
- Minimum Spanning Tree
- Planar Maximally-Filtered Graph ($\supset$ MST)
- Threshold filtering: drop all edges whose (absolute) weight is less than a certain threshold

In this project, we work with only PMFG. Threshold filtering could also be interesting, but the resultant graph might be reducible, which causes a handful of centrality measures and embedding algorithms to break down. Also, in the paper by Pozzi et al., the results they presented are based on PMFG, but they also claim that one can obtain similar results with MST. We decided to roll with PMFG, as it encapsulates more information than MST does.

## Centrality Measures

We take a page from your classical graph theory textbook: there are many ways to define what it means for a node (or an edge) to be a "central" node. The less central a node is, the less peripheral it is.

Below is an overview of some common graph centrality/peripherality measures.

#### Degree Centrality
The degree of a node can be considered a measure of centrality. Indeed, the higher the degree of a node, the more "important" a node is, and therefore it is considered more "central".

For a directed graph, one would need to make a distinction between in-degree and out-degree and therefore also the in/out-degree centrality. But for our project, the networks we consider are all undirected (as correlations are bi-directional).

#### Eigenvector Centrality
TODO: Spectral embedding & graph (adjacency) spectrum; quadratic maximisation problem; Perron-Frobenus on existence of unique sol (and PMFG by construction satisfies the assumptions (aperiodic irreducible/strongly-connected).

#### PageRank
Eigen centrality on in-degree-normalized adjacency matrix (i.e. the random walk matrix). Do note that the graphs we are considering in this project are all bi-directional and strongly connected. Hence these graphs are aperiodic-irreducible when plugged into PageRank, and all the Frobenius-Perron uniqueness/convergence holds nicely.

Pagerank gives a physical meaning to the eigenproblem: casting the eigendecomposition problem into a random walk problem
the eigenvector -- (unique) equilibrium random-walk distribution.

#### Eccentricity
The eccentricity of a vertex is the maximum distance between itself and any other vertex in the graph. The radius of a graph is its minimum eccentricity; the diameter of a graph is its maximum eccentricity. 

Eccentricity is a **peripherality** measure, not centrality measure. The higher the eccentricity of a node, the farther it is away it is from its "center", and therefore the more peripheral/less central it is. 

### Closeness Centrality
(assumes connecteness)

The farness of a vertex in a graph is the average of shortest paths between the said node and all other nodes. The closeness is the reciprocal of farness.

More "central" a node is ==> shorter its distance to all other vertices ==> lower farness/higher closeness

#### Betweenness Centrality
The ratio of shortest paths passing through a certain vertex, w.r.t. all shortest paths in the graph.

The more "central" a node is, the more likely a shortest path has to pass through it, and the higher its betweenness ratio is.

## Centrality Ranking
For each of these centrality measures, we can construct a centrality ranking of nodes by taking the descending order of a centrality measure (or, the ascending order of a peripheral measure). But each of these centrality measures are only good at identifying the most central nodes: the ordering of the top nodes are relatively robust to perturbations to the graph structure, but the same cannot be said for the rest of the nodes. Therefore, we aggregate these centrality rankings to construct a consolidated ranking.

## Centrality Ranking of Stock Correlation: Pipeline
We follow the pipeline from Pozzi et al.

Pipeline:
1. Construct correlation matrix
2. Construct dense network graph using off-diagonal correlation values as edge weights
5. Filter the dense network (PMFG/MST .etc)
4. Report centrality/peripherality measures for each node
5. Aggregate the measures; analyze and compose a hybrid centrality index


## Using Centrality Ranking as Trading Signals
Ideally, we want to update the centrality ranking as frequent as possible (say, daily), but our own implementation of PMFG takes a bit too long -- about one minute to parse each batch of data. So we can only recompute the rankings at a lower tick rate. We settled with a 3-month update.

We then try to trade using these signals. 

(TBC...)

